<a href="https://colab.research.google.com/github/RishabhGupta34/NLP_MODELS/blob/master/Word2Vec_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive

In [2]:
!pip install kaggle
!mkdir .kaggle
import json
token = {"username":"your_username","key":"your_key"}
with open('/content/.kaggle/kaggle.json', 'w') as file:
    json.dump(token, file)
!chmod 600 /content/.kaggle/kaggle.json
!cp /content/.kaggle/kaggle.json ~/.kaggle/kaggle.json
!kaggle config set -n path -v{/content}
!kaggle competitions download -c word2vec-nlp-tutorial

mkdir: cannot create directory ‘.kaggle’: File exists
- path is now set to: {/content}
  0% 0.00/276k [00:00<?, ?B/s]
100% 276k/276k [00:00<00:00, 87.0MB/s]
 69% 18.0M/26.0M [00:00<00:00, 58.8MB/s]
100% 26.0M/26.0M [00:00<00:00, 98.5MB/s]
 40% 5.00M/12.6M [00:00<00:00, 38.4MB/s]
100% 12.6M/12.6M [00:00<00:00, 61.7MB/s]
 39% 5.00M/13.0M [00:00<00:00, 30.6MB/s]
100% 13.0M/13.0M [00:00<00:00, 63.4MB/s]


In [3]:
!ls {/content}/competitions/word2vec-nlp-tutorial

labeledTrainData.tsv.zip  testData.tsv.zip
sampleSubmission.csv	  unlabeledTrainData.tsv.zip


In [4]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import os
import zipfile
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from sklearn.ensemble import  RandomForestClassifier
from gensim.models import word2vec
import logging
from gensim.models import Word2Vec
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',level=logging.INFO)
nltk.download('stopwords')
nltk.download('punkt')

/usr/local/lib/python3.6/dist-packages/smart_open/ssh.py:34: UserWarning: paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress
  warnings.warn('paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [0]:
zip_ref = zipfile.ZipFile('{/content}/competitions/word2vec-nlp-tutorial/labeledTrainData.tsv.zip', 'r')
zip_ref.extractall('./')
zip_ref.close()
zip_ref = zipfile.ZipFile('{/content}/competitions/word2vec-nlp-tutorial/unlabeledTrainData.tsv.zip', 'r')
zip_ref.extractall('./')
zip_ref.close()
zip_ref = zipfile.ZipFile('{/content}/competitions/word2vec-nlp-tutorial/testData.tsv.zip', 'r')
zip_ref.extractall('./')
zip_ref.close()

In [0]:
train_data=pd.read_csv('labeledTrainData.tsv',delimiter='\t',quoting=3)
test_data=pd.read_csv('testData.tsv',delimiter='\t',quoting=3)
un_train_data=pd.read_csv('unlabeledTrainData.tsv',delimiter='\t',quoting=3)

In [0]:
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

In [0]:
def clean_text(data,remove_stopwords=False):
  all_stop=stopwords.words('english')   #list of stopwords
  ss=SnowballStemmer("english")   #list of stemmer
  soup=BeautifulSoup(data)
  html_text=soup.get_text().lower()    #for removing html tags 
  slash=re.sub('[\\\\]',"",html_text)  # for removing backward slash
  quotes_text=re.sub('[\"¨]'," <QUOT> ",slash) # for removing double quotes
  com=re.sub('[,]'," <COM> ",quotes_text) # for removing comma
  apos=re.sub("[']"," <APOS> ",com) # for removing apostrophe
  dott=re.sub('\.+'," <DOT> ",apos)  # for removing full stop
  exc=re.sub('!+'," <EXC> ",dott)   # for removing exclamation
  num=re.sub("\d+"," <NUM> ",exc)  # for removing number
  brac=re.sub('[()]+'," <BRAC> ",num)  # for removing brackets
  ques=re.sub("\?+"," <QUES> ",brac)  # for removing question mark
  split_text=ques.split()  # for splitting the text
  if remove_stopwords:
    split_text=[w for w in split_text if w not in all_stop]  # for removing stopwords
  stem_text=[ss.stem(i) for i in split_text]  # for stemming each word
  return stem_text

In [0]:
def break_sent(train_data):
  tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
  cleaned_data=[]
  for i in range(train_data.shape[0]):
    raw_sentences=tokenizer.tokenize(train_data[i])
    for j in raw_sentences:
      if(len(j)>0):
        cleaned_data.append(clean_text(j))
  return cleaned_data

In [10]:
all_cleaned_sentences=[]
all_cleaned_sentences+=break_sent(train_data.review)
all_cleaned_sentences+=break_sent(un_train_data.review)

/usr/local/lib/python3.6/dist-packages/bs4/__init__.py:273: UserWarning: "b'.'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)
/usr/local/lib/python3.6/dist-packages/bs4/__init__.py:336: UserWarning: "http://www.happierabroad.com"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
/usr/local/lib/python3.6/dist-packages/bs4/__init__.py:336: UserWarning: "http://www.archive.org/details/LovefromaStranger"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
/usr/local/lib/python3.6/dist-packages/bs4/__init__.py:336: 

In [11]:
print(len(all_cleaned_sentences))

795538


In [0]:
num_features = 300    # Word vector dimensionality                      
min_word_count = 40   # Minimum word count                        
num_workers = 6       # Number of threads to run in parallel
context = 10          # Context window size                                                                                    
downsampling = 1e-3   # Downsample setting for frequent words

In [13]:
model = word2vec.Word2Vec(all_cleaned_sentences, workers=num_workers,size=num_features,min_count = min_word_count,window = context, sample = downsampling)

2019-04-27 22:09:28,673 : INFO : collecting all words and their counts
2019-04-27 22:09:28,675 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2019-04-27 22:09:28,754 : INFO : PROGRESS: at sentence #10000, processed 261919 words, keeping 14282 word types
2019-04-27 22:09:28,820 : INFO : PROGRESS: at sentence #20000, processed 523814 words, keeping 20411 word types
2019-04-27 22:09:28,887 : INFO : PROGRESS: at sentence #30000, processed 778285 words, keeping 25103 word types
2019-04-27 22:09:28,957 : INFO : PROGRESS: at sentence #40000, processed 1040208 words, keeping 29288 word types
2019-04-27 22:09:29,026 : INFO : PROGRESS: at sentence #50000, processed 1294552 words, keeping 32926 word types
2019-04-27 22:09:29,098 : INFO : PROGRESS: at sentence #60000, processed 1550740 words, keeping 36061 word types
2019-04-27 22:09:29,171 : INFO : PROGRESS: at sentence #70000, processed 1809233 words, keeping 39001 word types
2019-04-27 22:09:29,240 : INFO : PROGRESS:

In [0]:
model.init_sims(replace=True)   # init_sims will make the model much more memory-efficient.

model_name = "drive/300features_40minwords_10context"
model.save(model_name)

2019-04-26 19:05:11,947 : INFO : precomputing L2-norms of word weight vectors
2019-04-26 19:05:12,052 : INFO : saving Word2Vec object under drive/300features_40minwords_10context, separately None
2019-04-26 19:05:12,054 : INFO : not storing attribute vectors_norm
2019-04-26 19:05:12,056 : INFO : not storing attribute cum_table
2019-04-26 19:05:12,056 : WARNING : this function is deprecated, use smart_open.open instead
2019-04-26 19:05:18,240 : INFO : saved drive/300features_40minwords_10context


In [0]:
model=Word2Vec.load("drive/300features_40minwords_10context")

2019-04-26 20:22:47,231 : INFO : loading Word2Vec object from drive/300features_40minwords_10context
2019-04-26 20:22:47,232 : WARNING : this function is deprecated, use smart_open.open instead
2019-04-26 20:22:49,147 : INFO : loading wv recursively from drive/300features_40minwords_10context.wv.* with mmap=None
2019-04-26 20:22:49,148 : INFO : setting ignored attribute vectors_norm to None
2019-04-26 20:22:49,149 : INFO : loading vocabulary recursively from drive/300features_40minwords_10context.vocabulary.* with mmap=None
2019-04-26 20:22:49,155 : INFO : loading trainables recursively from drive/300features_40minwords_10context.trainables.* with mmap=None
2019-04-26 20:22:49,158 : INFO : setting ignored attribute cum_table to None
2019-04-26 20:22:49,160 : INFO : loaded drive/300features_40minwords_10context


In [0]:
def review_to_vector(reviews):
  cleaned_vectors=np.zeros((reviews.shape[0],model.wv.syn0.shape[1]))
  model_dict_words=model.wv.index2word
  for i in range(reviews.shape[0]):
    cleaned=clean_text(reviews[i],True)
    wc=0
    for word in cleaned:
      if word in model_dict_words:
        wc+=1
        cleaned_vectors[i]+=model[word]
    cleaned_vectors[i]/=wc
  return cleaned_vectors

In [15]:
train_average_vector_data=review_to_vector(train_data.review)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.wv.vectors instead).
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  # Remove the CWD from sys.path while we load stuff.


In [16]:
train_average_vector_data.shape

(25000, 300)

In [17]:
test_average_vector_data=review_to_vector(test_data.review)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.wv.vectors instead).
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  # Remove the CWD from sys.path while we load stuff.


In [0]:
rc=RandomForestClassifier(150)

In [19]:
rc.fit(train_average_vector_data,train_data.sentiment)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=150, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [0]:
test_prediction=rc.predict(test_average_vector_data)

In [0]:
output = pd.DataFrame( data={"id":test_data["id"], "sentiment":test_prediction} )

In [22]:
output.shape

(25000, 2)

In [23]:
output.head()

,id,sentiment
0,"""12311_10""",1
1,"""8348_2""",0
2,"""5828_4""",1
3,"""7186_2""",0
4,"""12128_7""",1


In [0]:
output.to_csv( "drive/Word2Vec_model.csv", index=False, quoting=3 )